In [1]:
from mido import MidiFile

import warnings
warnings.filterwarnings("ignore")

In [2]:
## Importa la tabla que parsea numeros de notas a nombre y octava
import pandas as pd
import numpy as np
import plotly.express as px
midi_notes = pd.read_csv('midi_notes.csv',sep=";")
midi_scales_chords = pd.read_csv('scales.csv',sep=";")

In [3]:
## Levanta el archivo midi, devuelve cantidad de tracks y mensajes contenidos
def load_midi_file(files_mid):
    mid = MidiFile(files_mid, clip=False)
    print(mid)
    print('segundos:', mid.length)
    return mid

In [4]:
## Itera sobre los mensaje midi de los tracks con mensjaes y genera un dataframe con notas, velocity, tick_start tick_stop
def get_theme_df(mid):
    dict_notes = {}
    dict_notes_end = {}
    dict_active_notes = {}
    count_notes = 0
    count_notes_end = 0
    last_note_on = 0
    info = []
    controls = []
    key_signatures = []
    time_signatures = []
    instrument_track = []
    n_track = 0
    n_tracks_used = 0
    for track in mid.tracks:

        track_name = track.name + str(n_track)
        info.append(track_name)
        n_track = n_track + 1 
        if len(track) > 100:      
            n_tracks_used = n_tracks_used + 1
            #print('mensajes:',len(track))
        time = 0
        has_note_off = any(msg.type == 'note_off' for msg in track)
        for msg in track:          
            if msg.type in ['note_on', 'note_off']:
                time = time + msg.time
                if (has_note_off and (msg.type == 'note_on')) or (not has_note_off and msg.velocity > 0):
                    dict_notes[count_notes] = {"note_num": msg.note, "start": time, "velocity": msg.velocity, "time_dif": time - last_note_on, "track_name": track_name}
                    dict_active_notes[msg.note] = time
                    count_notes = count_notes + 1
                    last_note_on = time
                else:
                    dict_notes_end[count_notes_end] = {"note_num": msg.note,"track_name": track_name, "start": dict_active_notes[msg.note], "end": time}
                    count_notes_end = count_notes_end + 1
            else:
                if (msg.type == 'control_change'):
                    controls.append(msg.value)
                elif (msg.type == 'key_signature'):
                    key_signatures.append(msg.key)
                elif (msg.type == 'time_signature'):
                    time_signatures.append(str(msg.numerator) + '/' + str(msg.denominator))
                elif (msg.type == 'program_change'):
                    instrument_track.append(str(n_track) + "-" + str(msg.program))
    tema_df = pd.DataFrame.from_dict(dict_notes, "index")
    tema_df_notes_end = pd.DataFrame.from_dict(dict_notes_end, "index")
    tema_df_merged = pd.merge(tema_df, tema_df_notes_end,on=['note_num','start','track_name'])
    controls = pd.Series(controls).head(40)
    controls = controls[controls > 10].sum() / n_tracks_used
    return tema_df_merged, info, controls, key_signatures, time_signatures, n_tracks_used, instrument_track                 

In [5]:
## Reemplaza outlyers de duración
def limit_outlyer_duration_notes(tema_df):
    notes_weight = pd.cut(tema_df.duration, 6)

    outlyeras_duration = pd.DataFrame(tema_df.duration.quantile([0.05,0.95]))

    mask_outlyers_lower = tema_df.duration < outlyeras_duration.duration[0.05]
    tema_df.loc[mask_outlyers_lower,'duration'] = outlyeras_duration.duration[0.05]

    mask_outlyers_higher = tema_df.duration > outlyeras_duration.duration[0.95]
    tema_df.loc[mask_outlyers_higher,'duration'] = outlyeras_duration.duration[0.95]

    notes_weight = pd.cut(tema_df.duration, 6)
    return tema_df

In [6]:
# Crea una mascara para limitar el ploteo en el tiempo
def plot_theme(data_to_plot, plot_from_second=0, plot_to_second=30, size_plot='duration', color_plot='grupos_track'):
    ## Poltea las notas en el tiempo
    mask_tiempo_from = (tema_df.segundo >= plot_from_second)
    mask_tiempo_to =  (tema_df.segundo <= plot_to_second)
    fig = px.scatter(data_frame = tema_df[mask_tiempo_from & mask_tiempo_to], x = "segundo", y = "note_num",
                     size = size_plot, color = color_plot, opacity = 1, color_discrete_sequence=None, labels='note', height=600,)

    fig.show()

In [7]:
## Reemplaza los tiempos de notas muy cercanas por notas simultaneas
def cuantize(tema_df):
    tema_df2_mask = (tema_df.iloc[1:len(tema_df),:].note_simultaneous == True)
    tema_df2_mask = tema_df2_mask
    tema_df2_mask[0] = False

    tema_df.loc[tema_df2_mask, 'start'] = np.NaN
    tema_df.start = tema_df.start.fillna(method='ffill')
    tema_df.start = tema_df.start.astype(int)
    return tema_df

In [8]:
def restore_df():
    tema_df = tema_df_copy

mid = load_midi_file('Symphony n2 op36 1mov.mid')
track_n = 0
for track in mid.tracks:   
    print('track:',track)
    for msg in track:   
        if msg.type in ['program_change']:
            print(msg.program)
    track_n = track_n + 1

In [9]:
def get_theme_stats(file_path, composer, file_name):
    ## Instancia el archivo midi
    mid = load_midi_file(file_path)
    duracion_tema = mid.length
    tema_df, info, controls, key_signatures, time_signatures, n_tracks_used, instrument_track = get_theme_df(mid)  

    ## Calcula la duración de cada nota
    tema_df.loc[:,'duration'] = tema_df.end - tema_df.start

    ## Parsea a enteros los valores numéricos
    for col in tema_df.loc[:,tema_df.columns!="track_name"].columns:
        tema_df[col] = pd.to_numeric(tema_df[col])

    ## agrega el nombre y octava de notas a la tabla
    tema_df = pd.merge(tema_df, midi_notes,how='left',left_on='note_num',right_on='note_number').drop('note_number',axis=1)

    ##elimina notas demasiado cortas y demasiado largas que pueden afectar al análisis
    limit_outlyer_duration_notes(tema_df)

    ## Categoriza duración

    cat_duration = pd.cut(tema_df.duration, 6, labels=['xxs','xs','s','m','l','xl'])
    tema_df.loc[:,'cat_duration'] = cat_duration

    ## Categoriza VELOCITY
    cat_velocity = pd.cut(tema_df.velocity, 6, labels=['pp','p','m','mf','f','ff'])
    tema_df.loc[:,'cat_velocity'] = cat_velocity

    ## Describe, muchos de estos valores van a ser utiles como predictores
    tema_describe = tema_df.describe()

    ## Cuantificación de notas:
    ## define cuales notas están tan cera que deberían estar en el mismo instante

    unit_duplicate = tema_describe.loc['min','duration'] / 1.5
    tema_df.loc[:,'note_simultaneous'] = tema_df.time_dif < unit_duplicate

    ## Reemplaza los tiempos de notas muy cercanas por notas simultaneas
    cuantize(tema_df)

    ## calcula la cantidad de ticks x segundo
    ticks_por_seg = tema_df.end.max() / duracion_tema

    ## Calcula la cantidad de notas que existen en simultaneo
    tema_df_simultaneous =  tema_df.start.value_counts()
    tema_df_simultaneous_times = tema_df_simultaneous.loc[tema_df_simultaneous > 1].index.to_list()

    tema_df.note_simultaneous = False
    tema_df.loc[:,'note_simultaneous'] = tema_df.start.apply(lambda x: True if x in tema_df_simultaneous_times else False) 

    ## Cada nota l y x l van a tener mas puntos que despues van a ser removidos
    tema_df.loc[:,'note_fake'] = 0
    long_notes_mask = tema_df.cat_duration.apply(lambda x: x in ['l','xl'])
    fake_notes = tema_df[long_notes_mask]
    fake_notes.start = fake_notes.apply(lambda x: x.start + (x.end - x.start) / 2 if x.cat_duration == 'l' else x.start + (x.end - x.start) / 3,axis=1)

    fake_notes.duration = fake_notes.duration / 2
    fake_notes.note_fake = 1

    xlong_notes_mask = tema_df.cat_duration == 'xl'
    fake_notes_lx = tema_df[xlong_notes_mask]
    fake_notes_lx.start = fake_notes_lx.start + (fake_notes_lx.end - fake_notes_lx.start)  / 3 * 2
    fake_notes_lx.duration = fake_notes_lx.duration / 2
    fake_notes_lx.note_fake = 1

    tema_df = pd.concat([tema_df,fake_notes, fake_notes_lx]).sort_values('start')

    ## Convierte unidad de medida de timpo Ticks a segundos en cada nota
    tema_df.loc[:,'segundo'] = tema_df.start / ticks_por_seg

    ## Implementacion DBSCAN para encontrar grupos de notas y segmentarlas
    from sklearn.cluster import DBSCAN
    from sklearn.preprocessing import StandardScaler

    X = tema_df[['start', 'note_num']]
    X.loc[:,'start'] = X.start / 15
    X.loc[:,'note_num'] = X.note_num / 0.2

    X = X.values

    # Implementación de DBSCAN
    dbscan = DBSCAN( eps=29, min_samples=3)
    y_dbscan = dbscan.fit_predict(X)
    y_dbscan

    labels = dbscan.labels_
    tema_df.loc[:,'grupos'] = labels
    cant_grupos = pd.Series(labels).unique().shape[0]

    ## Concatena los grupos creados con DBSCAN con las cadenas de notas recibidas en cada track
    tema_df.loc[:,'grupos_track'] = tema_df.grupos.astype(str) + "_" + tema_df.track_name
    #plot_theme(tema_df)

    ## elimina notas falsas
    notes_fake_mask = tema_df.note_fake == 1
    tema_df.drop(tema_df[notes_fake_mask].index, inplace=True)

    ## Imprime el dataset completo ordenado por grupos descartando el ruido
    ## Calcula el ruido
    mask_noise = tema_df.grupos != -1
    tema_df.sort_values(['grupos_track','start'],ascending=[True,True],inplace=True)

    ## Shape final del dataset
    notas_totales = tema_df.shape[0]

    ## indice de actividad (cantidad de notas) por tiempo
    cant_eventos_individuales = (notas_totales - len(tema_df_simultaneous_times) / 2)
    actividad_por_tiempo = cant_eventos_individuales / duracion_tema
    velocity_avg = tema_df.cat_velocity.value_counts(normalize=True)
    length_notes_avg = tema_df.cat_duration.value_counts(normalize=True)

    ## Analiza proporciones de notas y duraciones mas repetidas
    notes_weight = round(tema_df.note.value_counts(normalize=True) * 100,2)

    notes_weight = round(tema_df.note_name.value_counts(normalize=True) * 100,2)
    all_values_notes = pd.DataFrame(notes_weight).reset_index()
    most_probable_scale = all_values_notes.head(7)
    scale_coverage = notes_weight.head(7).sum()
    avr_vertical_notes = tema_df.note_simultaneous.sum() / notas_totales
    cant_pedal_sustain = controls
    cant_eventos_por_pedal = cant_eventos_individuales / cant_pedal_sustain if cant_pedal_sustain > 5 else np.NaN
    cant_pedales_seg = cant_pedal_sustain / duracion_tema if cant_pedal_sustain > 5 else np.NaN

    #obtiene informacion de la escala
    nombre_escala = pd.merge(most_probable_scale, midi_scales_chords, how='left', left_on='index', right_on='note_name')
    nombre_escala.fillna(0,inplace=True)
    nombre_escala_T = nombre_escala.T
    nombre_escala_T
    mask = nombre_escala_T.apply(lambda x: True if all(x != 0) else False, axis=1)
    mask
    tabla_esacla = nombre_escala_T[mask].T

    tabla_esacla

    nombre_columna_Tmaj = tabla_esacla.columns[3]
    tonalidad  = 0
    tonalidad_escala = 'M'
    dic = {6:1, 7:2, 1:3, 2:4, 3:5, 4:6, 5:7}
    if nombre_columna_Tmaj != "U":
        tabla_esacla.set_index(tabla_esacla.columns[3],inplace=True,drop=False)
        mayor_chord_coverage = tabla_esacla.loc[[1,3,5],:'note_name_x'].sum()[1]
        minor_chord_coverage = tabla_esacla.loc[[6,1,3],:'note_name_x'].sum()[1]


        if minor_chord_coverage > mayor_chord_coverage:
            tonalidad = tabla_esacla.iloc[0,2]
            # reemplazo el 6to grado por la tónica
             # define desired replacements here

            tabla_esacla[nombre_columna_Tmaj] = tabla_esacla[nombre_columna_Tmaj].apply(lambda x: dic[x])
            tabla_esacla
            tonalidad_escala = 'm'
            tabla_esacla.set_index(nombre_columna_Tmaj,inplace=True,drop=False)
            tabla_esacla.sort_index(inplace=True)
        else:
            tonalidad = nombre_columna_Tmaj

    elif len(key_signatures) > 0:
        if 'b' in key_signatures[0]:
            dict_keys = {'Db':'C#', 'Eb':'D#', 'Gb':'F#', 'Ab':'G#', 'Bb':'A#','Dbm':'C#m', 'Ebm':'D#m', 'Gbm':'F#m', 'Abm':'G#m', 'Bbm':'A#m'}
            tonalidad =  dict_keys[key_signatures[0]]
        else:
            tonalidad = key_signatures[0]

        midi_scales_chords_weighted = pd.merge(midi_scales_chords[['note_name', tonalidad]], all_values_notes, how='left', left_on='note_name', right_on='index',)
        midi_scales_chords_weighted.drop('index',axis=1,inplace=True)
        midi_scales_chords_weighted.columns = ['note_name', 'scale', 'weight']
        midi_scales_chords_weighted.set_index(midi_scales_chords_weighted.columns[1],inplace=True,drop=False)
        if 'm' in key_signatures[0]:
            mayor_chord_coverage = midi_scales_chords_weighted.loc[[3,5,7],'weight'].sum()
            minor_chord_coverage = midi_scales_chords_weighted.loc[[1,3,5],'weight'].sum()
        else:
            mayor_chord_coverage = midi_scales_chords_weighted.loc[[1,3,5],'weight'].sum()
            minor_chord_coverage = midi_scales_chords_weighted.loc[[6,1,3],'weight'].sum()        
            if minor_chord_coverage > mayor_chord_coverage:
                tonalidad_escala = 'm'
                midi_scales_chords_weighted_mask = midi_scales_chords_weighted.scale > 0
                midi_scales_chords_weighted.loc[midi_scales_chords_weighted_mask, 'scale'] = midi_scales_chords_weighted.loc[midi_scales_chords_weighted_mask, 'scale'].apply(lambda x: dic[x])
        midi_scales_chords_weighted.sort_index(inplace=True)
        midi_scales_chords_weighted.drop('scale',axis=1,inplace=True)
        scale_coverage = midi_scales_chords_weighted.head(7).sum()[1]
        tabla_esacla = midi_scales_chords_weighted
        tabla_esacla
    else:
        tonalidad = 'U'
        tonalidad_escala = 'U'
        tabla_esacla = pd.DataFrame(columns=['1','2'])
        tabla_esacla.iloc[0,7,:] = 0


    ## crea el Music stats    
    music_stats = pd.DataFrame(columns=['tonalidad','tonalidad_escala','scale_coverage','mayor_chord_coverage','minor_chord_coverage',
                                  'sc_1','sc_2','sc_3','sc_4','sc_5','sc_6','sc_7', 'avr_vertical_notes','cant_eventos_por_pedal',
                                  'cant_pedales_seg','duracion_seg','tracks_used','cant_grupos', 'info_tracks', 'midi_info',
                                    'compositor', 'file_name','instrument_track'])
    music_stats.loc[0] = [tonalidad, tonalidad_escala, scale_coverage,  mayor_chord_coverage, minor_chord_coverage, 
                   tabla_esacla.iloc[0,1], tabla_esacla.iloc[1,1], tabla_esacla.iloc[2,1], 
                   tabla_esacla.iloc[3,1], tabla_esacla.iloc[4,1], tabla_esacla.iloc[5,1], 
                   tabla_esacla.iloc[6,1], avr_vertical_notes,cant_eventos_por_pedal,
                   cant_pedales_seg,duracion_tema,n_tracks_used, cant_grupos,
                  " ".join(info), mid, composer, file_name, instrument_track]

    data_describe = pd.DataFrame(tema_describe.loc[tema_describe.index != 'count',['note_num','Octave','start','duration']].unstack())
    data_describe.reset_index(inplace=True)
    data_describe.loc[:,'name'] = data_describe.level_0 + "_" + data_describe.level_1
    data_describe.set_index('name',inplace=True)
    data_describe.drop(['level_0','level_1'],axis=1, inplace=True)
    df_final = pd.concat([music_stats.T,data_describe,velocity_avg,length_notes_avg])
    #display(df_final)
    return df_final.to_dict()

In [10]:
import os
bach = 'C:\DH\ds_blend_students_2020\desafio IV\MIDI folders\Greats\Greats\Classical Archives - The Greats (MIDI)\Bach'
mozart = 'C:\DH\ds_blend_students_2020\desafio IV\MIDI folders\Greats\Greats\Classical Archives - The Greats (MIDI)\Mozart'
vivaldi = 'C:\DH\ds_blend_students_2020\desafio IV\MIDI folders\Greats\Greats\Classical Archives - The Greats (MIDI)\Vivaldi'
schubert = 'C:\DH\ds_blend_students_2020\desafio IV\MIDI folders\Greats\Greats\Classical Archives - The Greats (MIDI)\Schubert'

# the dictionary to pass to panda's dataframe
dict = {}
count_files = 0
      
for root, dirs, files in os.walk(mozart, topdown=False):
    for name_file in files:
        dict[count_files] = {"file": os.path.join(root, name_file), "file_name": name_file, "compositor": 'Mozart'}
        count_files = count_files + 1

for root, dirs, files in os.walk(bach, topdown=False):
    for name_file in files:
        dict[count_files] = {"file": os.path.join(root, name_file), "file_name": name_file, "compositor": 'Bach'}
        count_files = count_files + 1
        
for root, dirs, files in os.walk(vivaldi, topdown=False):
    for name_file in files:
        dict[count_files] = {"file": os.path.join(root, name_file), "file_name": name_file, "compositor": 'Vivaldi'}
        count_files = count_files + 1      
        
for root, dirs, files in os.walk(schubert, topdown=False):
    for name_file in files:
        dict[count_files] = {"file": os.path.join(root, name_file), "file_name": name_file, "compositor": 'Schubert'}
        count_files = count_files + 1

df_files = pd.DataFrame.from_dict(dict, "index")
display(df_files)

,file,file_name,compositor
0,C:\DH\ds_blend_students_2020\desafio IV\MIDI f...,Piano Concerto n12 K414 1mov.mid,Mozart
1,C:\DH\ds_blend_students_2020\desafio IV\MIDI f...,Piano Concerto n12 K414 3mov.mid,Mozart
2,C:\DH\ds_blend_students_2020\desafio IV\MIDI f...,Piano Concerto n14 K449 1mov.mid,Mozart
3,C:\DH\ds_blend_students_2020\desafio IV\MIDI f...,Piano Concerto n14 K449 2mov.mid,Mozart
4,C:\DH\ds_blend_students_2020\desafio IV\MIDI f...,Piano Concerto n14 K449 3mov.mid,Mozart
...,...,...,...
1470,C:\DH\ds_blend_students_2020\desafio IV\MIDI f...,Symphony D944 n9 2mov ''Great''.mid,Schubert
1471,C:\DH\ds_blend_students_2020\desafio IV\MIDI f...,Symphony D944 n9 3mov ''Great''.mid,Schubert
1472,C:\DH\ds_blend_students_2020\desafio IV\MIDI f...,Symphony D944 n9 4mov ''Great''.mid,Schubert
1473,C:\DH\ds_blend_students_2020\desafio IV\MIDI f...,Valses Sentimentales op50 D779.mid,Schubert


In [11]:
dict = {}
count_files = 0

In [ ]:
from contextlib import suppress

errors_arr = []
for index, row in df_files.iterrows():
    with suppress(Exception):
        dict[count_files] = get_theme_stats(row.file, row.compositor, row.file_name)[0]
        count_files = count_files + 1
        print('success', str(count_files))

df_midi_stats = pd.DataFrame.from_dict(dict, "index")    


<midi file 'C:\\DH\\ds_blend_students_2020\\desafio IV\\MIDI folders\\Greats\\Greats\\Classical Archives - The Greats (MIDI)\\Mozart\\Piano Concertos\\Piano Concerto n12 K414 1mov.mid' type 1, 13 tracks, 17316 messages>
segundos: 635.7304227166791
success 1
<midi file 'C:\\DH\\ds_blend_students_2020\\desafio IV\\MIDI folders\\Greats\\Greats\\Classical Archives - The Greats (MIDI)\\Mozart\\Piano Concertos\\Piano Concerto n12 K414 3mov.mid' type 1, 14 tracks, 10689 messages>
segundos: 399.70017821666585
success 2
<midi file 'C:\\DH\\ds_blend_students_2020\\desafio IV\\MIDI folders\\Greats\\Greats\\Classical Archives - The Greats (MIDI)\\Mozart\\Piano Concertos\\Piano Concerto n14 K449 1mov.mid' type 1, 10 tracks, 17746 messages>
segundos: 454.4864088749969
success 3
<midi file 'C:\\DH\\ds_blend_students_2020\\desafio IV\\MIDI folders\\Greats\\Greats\\Classical Archives - The Greats (MIDI)\\Mozart\\Piano Concertos\\Piano Concerto n14 K449 2mov.mid' type 1, 10 tracks, 6659 messages>
segund

segundos: 553.9453518958302
success 31
<midi file "C:\\DH\\ds_blend_students_2020\\desafio IV\\MIDI folders\\Greats\\Greats\\Classical Archives - The Greats (MIDI)\\Mozart\\Piano Concertos\\Piano Concerto n7 for 3 Pianos K242 2mov ''Lodron''.mid" type 1, 21 tracks, 13150 messages>
segundos: 501.8970850145802
success 32
<midi file "C:\\DH\\ds_blend_students_2020\\desafio IV\\MIDI folders\\Greats\\Greats\\Classical Archives - The Greats (MIDI)\\Mozart\\Piano Concertos\\Piano Concerto n7 for 3 Pianos K242 3mov ''Lodron''.mid" type 1, 21 tracks, 12437 messages>
segundos: 337.4239356291677
success 33
<midi file 'C:\\DH\\ds_blend_students_2020\\desafio IV\\MIDI folders\\Greats\\Greats\\Classical Archives - The Greats (MIDI)\\Mozart\\Piano Concertos\\Piano Concerto n8 K246 1mov.mid' type 1, 13 tracks, 14529 messages>
segundos: 423.0922855312444
success 34
<midi file 'C:\\DH\\ds_blend_students_2020\\desafio IV\\MIDI folders\\Greats\\Greats\\Classical Archives - The Greats (MIDI)\\Mozart\\Piano

segundos: 748.3818301416131
success 62
<midi file 'C:\\DH\\ds_blend_students_2020\\desafio IV\\MIDI folders\\Greats\\Greats\\Classical Archives - The Greats (MIDI)\\Mozart\\Piano Sonatas\\Nueva carpeta\\K358 Piano Sonata 4 Hands.mid' type 1, 8 tracks, 15916 messages>
segundos: 667.1999999999744
success 63
<midi file 'C:\\DH\\ds_blend_students_2020\\desafio IV\\MIDI folders\\Greats\\Greats\\Classical Archives - The Greats (MIDI)\\Mozart\\Piano Sonatas\\Nueva carpeta\\K381 Piano Sonata 4 Hands.mid' type 1, 8 tracks, 17553 messages>
segundos: 621.2518358916753
success 64
<midi file 'C:\\DH\\ds_blend_students_2020\\desafio IV\\MIDI folders\\Greats\\Greats\\Classical Archives - The Greats (MIDI)\\Mozart\\Piano Sonatas\\Nueva carpeta\\K457 Piano Sonata n14 .mid' type 0, 1 tracks, 16564 messages>
segundos: 911.1312499999752
success 65
<midi file 'C:\\DH\\ds_blend_students_2020\\desafio IV\\MIDI folders\\Greats\\Greats\\Classical Archives - The Greats (MIDI)\\Mozart\\Piano Sonatas\\Nueva carpe

segundos: 297.1305649999999
success 95
<midi file 'C:\\DH\\ds_blend_students_2020\\desafio IV\\MIDI folders\\Greats\\Greats\\Classical Archives - The Greats (MIDI)\\Mozart\\Symphonies\\Symphony n25 K183 3mov.mid' type 1, 12 tracks, 6191 messages>
segundos: 220.41200439999992
success 96
<midi file 'C:\\DH\\ds_blend_students_2020\\desafio IV\\MIDI folders\\Greats\\Greats\\Classical Archives - The Greats (MIDI)\\Mozart\\Symphonies\\Symphony n25 K183 4mov.mid' type 1, 11 tracks, 15181 messages>
segundos: 331.68942699999826
success 97
<midi file 'C:\\DH\\ds_blend_students_2020\\desafio IV\\MIDI folders\\Greats\\Greats\\Classical Archives - The Greats (MIDI)\\Mozart\\Symphonies\\Symphony n28 K200 1mov.mid' type 1, 15 tracks, 14439 messages>
segundos: 305.48843283333144
success 98
<midi file 'C:\\DH\\ds_blend_students_2020\\desafio IV\\MIDI folders\\Greats\\Greats\\Classical Archives - The Greats (MIDI)\\Mozart\\Symphonies\\Symphony n28 K200 2mov.mid' type 1, 13 tracks, 4942 messages>
segundo

segundos: 710.6600678667311
success 128
<midi file "C:\\DH\\ds_blend_students_2020\\desafio IV\\MIDI folders\\Greats\\Greats\\Classical Archives - The Greats (MIDI)\\Mozart\\Symphonies\\Symphony n38 K504 2mov ''Prague''.mid" type 1, 10 tracks, 13545 messages>
segundos: 622.8123435250038
success 129
<midi file "C:\\DH\\ds_blend_students_2020\\desafio IV\\MIDI folders\\Greats\\Greats\\Classical Archives - The Greats (MIDI)\\Mozart\\Symphonies\\Symphony n38 K504 3mov ''Prague''.mid" type 1, 12 tracks, 16038 messages>
segundos: 223.46456658333085
success 130
<midi file 'C:\\DH\\ds_blend_students_2020\\desafio IV\\MIDI folders\\Greats\\Greats\\Classical Archives - The Greats (MIDI)\\Mozart\\Symphonies\\Symphony n39 K543 1mov.mid' type 1, 14 tracks, 32372 messages>
segundos: 645.9758230166483
success 131
<midi file 'C:\\DH\\ds_blend_students_2020\\desafio IV\\MIDI folders\\Greats\\Greats\\Classical Archives - The Greats (MIDI)\\Mozart\\Symphonies\\Symphony n39 K543 2mov.mid' type 1, 14 track

success 160
<midi file 'C:\\DH\\ds_blend_students_2020\\desafio IV\\MIDI folders\\Greats\\Greats\\Classical Archives - The Greats (MIDI)\\Mozart\\The Chelsea Notebook\\Early Pieces n6 Llittle minuet.mid' type 1, 18 tracks, 313 messages>
segundos: 26.098847099999993
success 161
<midi file 'C:\\DH\\ds_blend_students_2020\\desafio IV\\MIDI folders\\Greats\\Greats\\Classical Archives - The Greats (MIDI)\\Mozart\\The Chelsea Notebook\\Early Pieces n7 Country dance.mid' type 1, 4 tracks, 709 messages>
segundos: 66.45762585000024
success 162
<midi file 'C:\\DH\\ds_blend_students_2020\\desafio IV\\MIDI folders\\Greats\\Greats\\Classical Archives - The Greats (MIDI)\\Mozart\\The Chelsea Notebook\\Early Pieces n8 Minuet.mid' type 1, 18 tracks, 710 messages>
segundos: 56.28407864166687
success 163
<midi file 'C:\\DH\\ds_blend_students_2020\\desafio IV\\MIDI folders\\Greats\\Greats\\Classical Archives - The Greats (MIDI)\\Mozart\\The Chelsea Notebook\\Early Pieces n9 Allegretto.mid' type 1, 18 tra

success 194
<midi file 'C:\\DH\\ds_blend_students_2020\\desafio IV\\MIDI folders\\Greats\\Greats\\Classical Archives - The Greats (MIDI)\\Mozart\\K393 Solfeggi n1.mid' type 1, 4 tracks, 3406 messages>
segundos: 199.9749984374985
success 195
<midi file 'C:\\DH\\ds_blend_students_2020\\desafio IV\\MIDI folders\\Greats\\Greats\\Classical Archives - The Greats (MIDI)\\Mozart\\K393 Solfeggi n2.mid' type 1, 4 tracks, 894 messages>
segundos: 134.48871765625026
success 196
<midi file 'C:\\DH\\ds_blend_students_2020\\desafio IV\\MIDI folders\\Greats\\Greats\\Classical Archives - The Greats (MIDI)\\Mozart\\K393 Solfeggi n3.mid' type 1, 4 tracks, 3206 messages>
segundos: 182.0414863645825
success 197
<midi file 'C:\\DH\\ds_blend_students_2020\\desafio IV\\MIDI folders\\Greats\\Greats\\Classical Archives - The Greats (MIDI)\\Mozart\\K393 Solfeggi n4.mid' type 1, 3 tracks, 877 messages>
segundos: 81.89583333333333
success 198
<midi file 'C:\\DH\\ds_blend_students_2020\\desafio IV\\MIDI folders\\Gre

from contextlib import suppress

errors_arr = []
for index, row in df_files.iterrows():
    dict[count_files] = get_theme_stats(row.file, row.compositor, row.file_name)[0]
    count_files = count_files + 1

df_midi_stats = pd.DataFrame.from_dict(dict, "index")   

df_midi_stats

In [ ]:
df_midi_stats.to_csv('midit_stats.csv')

In [ ]:
df_midi_stats